In [1]:
from importlib.metadata import version

In [2]:
from article_dataset_builder.harvest import Harverster
from software_mentions_client.client import software_mentions_client
import requests
import tqdm

In [3]:
version('article_dataset_builder')

'0.2.6'

In [4]:
version('software_mentions_client')

'0.1.8'

In [5]:
version('lmdb')

'1.4.1'

In [7]:
# Patrice pointed out the input file is pcmids not DOIs.
#https://github.com/howisonlab/screenit-softcite/issues/1
# First convert file so that it is one pmcid per line.
!cut -d ',' -f1 config/comparison_full_set.csv | tail -n +2 > config/comparison_full_set-pmcids.csv

In [11]:
!mkdir ./resources # seemingly require by the harvester.
harvester = Harverster(config_path="config/config-harvester.json", dump_metadata=True)
# harvester.reset(True)

  1%|          | 46036/5454346 [00:00<00:11, 460283.62it/s]

building PMC resource map - done only one time


100%|██████████| 5454346/5454346 [00:21<00:00, 257965.18it/s]


In [16]:
harvester.harvest_pmcids("config/comparison_full_set-pmcids.csv")

  0%|          | 0/1500 [00:00<?, ?it/s]

number of articles to harvest: 1500 



InvalidParameterError: mdb_txn_begin: Invalid argument

In [12]:
harvester.diagnostic(full=True)

---
total entries: 0
---
total fully successful entries: 0 entries with valid OA URL and PDF and TEI XML
---
total invalid OA URL: 0
total entries with valid OA URL: 0
---
total invalid PDF: 0
total entries with successfully downloaded PDF: 0
---
total invalid TEI: 0
total entries with successfully converted TEI XML: 0
---
total identifiers: 0
total missing entries in metadata map: 0
---
total entries with GROBID TEI file: 0
total entries with Pub2TEI TEI file: 0
total entries with at least one TEI file: 0
---


In [47]:
from article_dataset_builder.nlm2tei import Nlm2tei
nlm2tei = Nlm2tei(config_path="./config/config-harvester.json")
nlm2tei.process()

Successfully created the directory ./data/pub2tei_tmp


Saxon-HE 9.9.0.2J from Saxonica
Java version 11.0.19
Using parser com.sun.org.apache.xerces.internal.jaxp.SAXParserImpl$JAXPSAXParser
URIResolver.resolve href="Imports.xsl" base="file:/home/jlh5498/Pub2TEI/Stylesheets/Publishers.xsl"
URIResolver.resolve href="Default.xsl" base="file:/home/jlh5498/Pub2TEI/Stylesheets/Imports.xsl"
URIResolver.resolve href="NameComponents.xsl" base="file:/home/jlh5498/Pub2TEI/Stylesheets/Imports.xsl"
URIResolver.resolve href="JournalComponents.xsl" base="file:/home/jlh5498/Pub2TEI/Stylesheets/Imports.xsl"
URIResolver.resolve href="BookComponents.xsl" base="file:/home/jlh5498/Pub2TEI/Stylesheets/Imports.xsl"
URIResolver.resolve href="NamesDatesPlaces.xsl" base="file:/home/jlh5498/Pub2TEI/Stylesheets/Imports.xsl"
URIResolver.resolve href="KeywordsAbstract.xsl" base="file:/home/jlh5498/Pub2TEI/Stylesheets/Imports.xsl"
URIResolver.resolve href="FullTextTags.xsl" base="file:/home/jlh5498/Pub2TEI/Stylesheets/Imports.xsl"
URIResolver.resolve href="ISOifiers.xsl"

e.returncode 2
e.output None

runtime: 8.958 seconds 


Stylesheet compilation time: 8.235303754s (8235.303754ms)
Processing file:/home/jlh5498/./data/pub2tei_tmp/JATS-archivearticle1-3-mathml3.dtd
Using parser com.sun.org.apache.xerces.internal.jaxp.SAXParserImpl$JAXPSAXParser
Building tree for file:/home/jlh5498/./data/pub2tei_tmp/JATS-archivearticle1-3-mathml3.dtd using class net.sf.saxon.tree.tiny.TinyBuilder
Error on line 1 column 1 of JATS-archivearticle1-3-mathml3.dtd:
  SXXP0003: Error reported by XML parser: Premature end of file.
While processing JATS-archivearticle1-3-mathml3.dtd: org.xml.sax.SAXParseException; systemId: file:/home/jlh5498/./data/pub2tei_tmp/JATS-archivearticle1-3-mathml3.dtd; lineNumber: 1; columnNumber: 1; Premature end of file.

Processing file:/home/jlh5498/./data/pub2tei_tmp/JATS-archivearticle1-mathml3.dtd
Using parser com.sun.org.apache.xerces.internal.jaxp.SAXParserImpl$JAXPSAXParser
Building tree for file:/home/jlh5498/./data/pub2tei_tmp/JATS-archivearticle1-mathml3.dtd using class net.sf.saxon.tree.tiny

In [ ]:
client = software_mentions_client(config_path="config/config-client.json")
client.annotate_collection("./data", force=True)

In [38]:
from pathlib import Path
import pandas as pd

all_paths = list(Path(".data").rglob("*.pdf"))
len(all_paths)
software_paths = list(Path(".data").rglob("*.software.json"))
#len(paths)

In [40]:
len(all_paths)

1432

In [44]:
software_paths

[PosixPath('.data/00/0d/0e/a9/000d0ea9-f5fa-4de4-9444-c673ffdea97a/000d0ea9-f5fa-4de4-9444-c673ffdea97a.software.json'),
 PosixPath('.data/00/3e/f5/48/003ef548-e101-4991-a4e2-f23804aaa8a2/003ef548-e101-4991-a4e2-f23804aaa8a2.software.json'),
 PosixPath('.data/00/46/54/f9/004654f9-f07c-49f6-9e3f-5b10a6e03531/004654f9-f07c-49f6-9e3f-5b10a6e03531.software.json'),
 PosixPath('.data/00/85/b9/da/0085b9da-3407-4571-84c9-ec8b18b68389/0085b9da-3407-4571-84c9-ec8b18b68389.software.json'),
 PosixPath('.data/00/ae/0a/c4/00ae0ac4-b48a-47eb-85e1-c61b07aeb63d/00ae0ac4-b48a-47eb-85e1-c61b07aeb63d.software.json'),
 PosixPath('.data/00/b1/22/a5/00b122a5-a800-4618-8058-075d7b21f8f8/00b122a5-a800-4618-8058-075d7b21f8f8.software.json'),
 PosixPath('.data/00/cb/96/d7/00cb96d7-ec0c-42ff-a8f3-3fc0b50564e9/00cb96d7-ec0c-42ff-a8f3-3fc0b50564e9.software.json'),
 PosixPath('.data/00/d8/db/84/00d8db84-1379-4fff-bd51-2c64fb2a2b37/00d8db84-1379-4fff-bd51-2c64fb2a2b37.software.json'),
 PosixPath('.data/01/14/4f/45/01

In [45]:

df = pd.DataFrame([pd.read_json(p, typ="series") for p in software_paths])
df['original_file_path']

0     ./data/00/0d/0e/a9/000d0ea9-f5fa-4de4-9444-c67...
1                                                   NaN
2                                                   NaN
3     ./data/00/85/b9/da/0085b9da-3407-4571-84c9-ec8...
4     ./data/00/ae/0a/c4/00ae0ac4-b48a-47eb-85e1-c61...
5                                                   NaN
6     ./data/00/cb/96/d7/00cb96d7-ec0c-42ff-a8f3-3fc...
7     ./data/00/d8/db/84/00d8db84-1379-4fff-bd51-2c6...
8                                                   NaN
9                                                   NaN
10    ./data/01/4b/cd/8a/014bcd8a-b6c5-45ae-b205-eed...
11    ./data/01/56/62/9f/0156629f-84f4-4ca6-88b6-fa6...
12    ./data/01/6f/e7/7a/016fe77a-e398-4a73-9c76-55b...
Name: original_file_path, dtype: object

In [31]:
df['mentions'].apply(len)

0     1
1     0
2     0
3     2
4     1
5     0
6     5
7     7
8     0
9     0
10    1
11    7
12    2
Name: mentions, dtype: int64

In [43]:
df.explode("mentions")

,application,version,date,md5,pages,mentions,references,runtime,id,metadata,original_file_path,file_name
0,software-mentions,0.8.0-SNAPSHOT,2023-07-12T22:05+0000,0BA63CEAEB3AADB5900CA135CA9A16DF,"[{'page_height': 793.701, 'page_width': 595.27...","{'type': 'software', 'software-type': 'environ...",[],30896,000d0ea9-f5fa-4de4-9444-c673ffdea97a,{'URL': 'http://dx.doi.org/10.1016/j.jtumed.20...,./data/00/0d/0e/a9/000d0ea9-f5fa-4de4-9444-c67...,000d0ea9-f5fa-4de4-9444-c673ffdea97a.pdf
1,software-mentions,0.8.0-SNAPSHOT,2023-07-12T22:03+0000,D018893C82450215DEC10357743A4607,"[{'page_height': 782.986, 'page_width': 602.98...",NaN,[],14726,NaN,NaN,NaN,NaN
2,software-mentions,0.8.0-SNAPSHOT,2023-07-12T22:03+0000,95080879D939D9CC0A3EE1E9F49EB4C8,"[{'page_height': 793.701, 'page_width': 595.27...",NaN,[],12380,NaN,NaN,NaN,NaN
3,software-mentions,0.8.0-SNAPSHOT,2023-07-12T22:03+0000,50788C9227CEC46B96C5AF5C51980B9E,"[{'page_height': 792.0, 'page_width': 612.0}, ...","{'type': 'software', 'software-type': 'softwar...",[],24871,0085b9da-3407-4571-84c9-ec8b18b68389,{'URL': 'http://dx.doi.org/10.1016/j.ijrobp.20...,./data/00/85/b9/da/0085b9da-3407-4571-84c9-ec8...,0085b9da-3407-4571-84c9-ec8b18b68389.pdf
3,software-mentions,0.8.0-SNAPSHOT,2023-07-12T22:03+0000,50788C9227CEC46B96C5AF5C51980B9E,"[{'page_height': 792.0, 'page_width': 612.0}, ...","{'type': 'software', 'software-type': 'softwar...",[],24871,0085b9da-3407-4571-84c9-ec8b18b68389,{'URL': 'http://dx.doi.org/10.1016/j.ijrobp.20...,./data/00/85/b9/da/0085b9da-3407-4571-84c9-ec8...,0085b9da-3407-4571-84c9-ec8b18b68389.pdf
4,software-mentions,0.8.0-SNAPSHOT,2023-07-12T22:03+0000,99FF67B66E9EA6E973D9E1BB76F30022,"[{'page_height': 792.0, 'page_width': 612.0}, ...","{'type': 'software', 'software-type': 'environ...",[],10546,00ae0ac4-b48a-47eb-85e1-c61b07aeb63d,{'URL': 'http://dx.doi.org/10.24926/iip.v11i4....,./data/00/ae/0a/c4/00ae0ac4-b48a-47eb-85e1-c61...,00ae0ac4-b48a-47eb-85e1-c61b07aeb63d.pdf
5,software-mentions,0.8.0-SNAPSHOT,2023-07-12T22:03+0000,6775E23B4DD836AF9E467E6C43EFB1F8,"[{'page_height': 793.701, 'page_width': 595.27...",NaN,[],10952,NaN,NaN,NaN,NaN
6,software-mentions,0.8.0-SNAPSHOT,2023-07-12T22:04+0000,F38AA431526555EDEE4484BCCDA54745,"[{'page_height': 792.0, 'page_width': 612.0}, ...","{'type': 'software', 'software-type': 'environ...",[],17716,00cb96d7-ec0c-42ff-a8f3-3fc0b50564e9,{'URL': 'http://dx.doi.org/10.2147/jmdh.s31263...,./data/00/cb/96/d7/00cb96d7-ec0c-42ff-a8f3-3fc...,00cb96d7-ec0c-42ff-a8f3-3fc0b50564e9.pdf
6,software-mentions,0.8.0-SNAPSHOT,2023-07-12T22:04+0000,F38AA431526555EDEE4484BCCDA54745,"[{'page_height': 792.0, 'page_width': 612.0}, ...","{'type': 'software', 'software-type': 'environ...",[],17716,00cb96d7-ec0c-42ff-a8f3-3fc0b50564e9,{'URL': 'http://dx.doi.org/10.2147/jmdh.s31263...,./data/00/cb/96/d7/00cb96d7-ec0c-42ff-a8f3-3fc...,00cb96d7-ec0c-42ff-a8f3-3fc0b50564e9.pdf
6,software-mentions,0.8.0-SNAPSHOT,2023-07-12T22:04+0000,F38AA431526555EDEE4484BCCDA54745,"[{'page_height': 792.0, 'page_width': 612.0}, ...","{'type': 'software', 'software-type': 'softwar...",[],17716,00cb96d7-ec0c-42ff-a8f3-3fc0b50564e9,{'URL': 'http://dx.doi.org/10.2147/jmdh.s31263...,./data/00/cb/96/d7/00cb96d7-ec0c-42ff-a8f3-3fc...,00cb96d7-ec0c-42ff-a8f3-3fc0b50564e9.pdf
